In [ ]:
#Investment projection

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- INPUTS ---
current_value = 2686638   # ₹26.86 lakh current corpus
monthly_sip = 35000       # current SIP
annual_stepup = 0.10      # 10% step-up per year until May 2030
new_sip_2030 = 100000     # SIP after May 2030
return_rate = 0.12        # expected CAGR
start_year = 2025
end_year = 2055

# --- CALCULATION SETUP ---
months = (end_year - start_year + 1) * 12
monthly_rate = (1 + return_rate) ** (1/12) - 1

corpus = []
years = []

value = current_value
current_sip = monthly_sip

for month in range(months):
    year = start_year + month // 12
    years.append(year)
    
    # SIP logic
    if year < 2030:
        current_sip = monthly_sip * ((1 + annual_stepup) ** (year - start_year))
    else:
        current_sip = new_sip_2030
    
    # Apply SIP + monthly compounding
    value = (value + current_sip) * (1 + monthly_rate)
    corpus.append(value)

# --- YEARLY SUMMARY ---
df = pd.DataFrame({"Year": years, "Corpus": corpus})
annual_df = df.groupby("Year")["Corpus"].last().reset_index()
annual_df["Corpus (₹ Cr)"] = annual_df["Corpus"] / 1e7

# --- FIND MILESTONES ---
milestones = {}
for target in [1e7, 5e7, 1e8, 10e8]:
    hit = annual_df[annual_df["Corpus"] >= target]
    if not hit.empty:
        milestones[f"₹{target/1e7:.0f} Cr"] = int(hit.iloc[0]["Year"])
    else:
        milestones[f"₹{target/1e7:.0f} Cr"] = None

# --- DISPLAY TABLE ---
print("\nYear-by-Year Projection (Values in ₹ Crore):\n")
print(annual_df.to_string(index=False))

print("\nMilestone Years:")
for k, v in milestones.items():
    print(f"{k}: {v if v else 'Not reached by 2055'}")

# --- VISUAL CHART ---
plt.figure(figsize=(10,6))
plt.plot(annual_df["Year"], annual_df["Corpus (₹ Cr)"], marker="o", color="#0072B2")
plt.axhline(100, color="red", linestyle="--", label="₹100 Crore Target")
plt.title("Projected Portfolio Growth (with Step-up + ₹1L SIP from 2030)", fontsize=13)
plt.xlabel("Year")
plt.ylabel("Corpus (₹ Crore)")
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- INPUTS ---
current_value = 2686638
monthly_sip = 35000
annual_stepup = 0.10       # till 2030
new_sip_2030 = 100000
start_year = 2025
end_year = 2055
returns = [0.12, 0.15]     # 12% and 15%

def simulate_growth(return_rate):
    months = (end_year - start_year + 1) * 12
    monthly_rate = (1 + return_rate) ** (1/12) - 1
    corpus, years = [], []
    value = current_value

    for month in range(months):
        year = start_year + month // 12
        years.append(year)
        # SIP logic
        if year < 2030:
            sip = monthly_sip * ((1 + annual_stepup) ** (year - start_year))
        else:
            sip = new_sip_2030
        value = (value + sip) * (1 + monthly_rate)
        corpus.append(value)

    df = pd.DataFrame({"Year": years, "Corpus": corpus})
    return df.groupby("Year")["Corpus"].last().reset_index()

# --- RUN BOTH SCENARIOS ---
df_12 = simulate_growth(0.12)
df_15 = simulate_growth(0.15)

# --- MERGE RESULTS ---
merged = df_12.merge(df_15, on="Year", suffixes=("_12%", "_15%"))
merged["Corpus (Cr) 12%"] = merged["Corpus_12%"] / 1e7
merged["Corpus (Cr) 15%"] = merged["Corpus_15%"] / 1e7

# --- MILESTONES ---
milestones = {}
for target, label in zip([1e7, 1e8, 10e8], ["₹1 Cr", "₹10 Cr", "₹100 Cr"]):
    yr12 = merged.loc[merged["Corpus_12%"] >= target, "Year"].min()
    yr15 = merged.loc[merged["Corpus_15%"] >= target, "Year"].min()
    milestones[label] = (yr12, yr15)

# --- DISPLAY TABLE ---
print("Milestone Comparison:\n")
print("Target\t12% CAGR\t15% CAGR")
for label, (y12, y15) in milestones.items():
    print(f"{label}\t{y12}\t\t{y15}")

# --- PLOT ---
plt.figure(figsize=(10,6))
plt.plot(merged["Year"], merged["Corpus (Cr) 12%"], label="12% CAGR", color="#0072B2", linewidth=2)
plt.plot(merged["Year"], merged["Corpus (Cr) 15%"], label="15% CAGR", color="#009E73", linewidth=2)
plt.axhline(100, color="red", linestyle="--", label="₹100 Cr target")
plt.title("Portfolio Growth: 12% vs 15% CAGR (with 10% Step-up + ₹1L SIP from 2030)", fontsize=12)
plt.xlabel("Year")
plt.ylabel("Corpus (₹ Crore)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()
